In [3]:
import cv2
import numpy as np
from pathlib import Path
import sys

# ==========================================================
# ===== TIỀN XỬ LÝ (GRAYSCALE + BLUR)
# ==========================================================
def tien_xu_ly(image):
    #Chuyển ảnh sang ảnh xám và làm mờ Gaussian.
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image
    blurred_image = cv2.GaussianBlur(gray_image, (15, 15), 0)
    return blurred_image

# ==========================================================
# =====  KẾT NỐI CẠNH (MORPHOLOGY)
# ==========================================================
def ket_noi_canh(edge_img, kernel_size=(9, 5), shape='rect',
                 do_close=True, do_dilate=False, iterations=1):

    # Áp dụng các phép toán hình thái (Morphological Operations) để kết nối các cạnh bị đứt gãy từ ảnh Canny.
    if edge_img is None:
        raise ValueError("Ảnh đầu vào rỗng hoặc không đọc được!")

    # Chuyển về nhị phân 0/255 (đảm bảo dạng chuẩn)
    if len(edge_img.shape) == 3:
        gray = cv2.cvtColor(edge_img, cv2.COLOR_BGR2GRAY)
    else:
        gray = edge_img.copy()
    binary = np.where(gray > 0, 255, 0).astype('uint8')

    # Tạo kernel (structuring element) theo lựa chọn
    if shape == 'rect':
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
    elif shape == 'ellipse':
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    elif shape == 'cross':
        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, kernel_size)
    else:
        raise ValueError("shape phải là 'rect', 'ellipse' hoặc 'cross'!")

    # Áp dụng phép đóng (closing = dilate -> erode)
    result = binary.copy()
    if do_close:
        result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel, iterations=iterations)

    # Áp dụng phép giãn nở (dilate)
    if do_dilate:
        result = cv2.dilate(result, kernel, iterations=iterations)

    # Chuẩn hóa đầu ra về 0/255
    result = np.where(result > 0, 255, 0).astype('uint8')
    return result

# ==========================================================
# ===== TÌM ỨNG CỬ VIÊN (FIND CONTOURS)
# ==========================================================
def tim_ung_cu_vien(anh_nhi_phan: np.ndarray):
    if len(anh_nhi_phan.shape) == 3:
        gray = cv2.cvtColor(anh_nhi_phan, cv2.COLOR_BGR2GRAY)
    else:
        gray = anh_nhi_phan.copy()
        
    # Nhị phân 0/255 
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # Tìm contours
    found = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = found[1] if len(found) == 3 else found[0]
    return contours, binary

# ==========================================================
# ===== LỌC ỨNG CỬ VIÊN
# ==========================================================
def loc_theo_dien_tich(contours, min_area, max_area):
    filtered_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if min_area < area < max_area:
            filtered_contours.append(cnt)
    return filtered_contours

def loc_theo_ty_le_khung_hinh(contours, min_aspect_ratio_vuong=1.5, max_aspect_ratio_vuong=2.5,
                             min_aspect_ratio_dai=3.5, max_aspect_ratio_dai=5.0):
    ung_cu_vien_cuoi_cung = []
    for cnt in contours:
        # Lấy bounding box (hình chữ nhật bao quanh)
        x, y, w, h = cv2.boundingRect(cnt)
        # Tránh lỗi chia cho 0
        if h == 0:
            continue
        ty_le = float(w) / h
        
        # Kiểm tra xem có nằm trong ngưỡng của biển vuông (2 dòng)
        la_bien_vuong = (min_aspect_ratio_vuong < ty_le < max_aspect_ratio_vuong)
        
        # Kiểm tra xem có nằm trong ngưỡng của biển dài (1 dòng)
        la_bien_dai = (min_aspect_ratio_dai < ty_le < max_aspect_ratio_dai)
        
        if la_bien_vuong or la_bien_dai:
            ung_cu_vien_cuoi_cung.append(cnt)
            
    return ung_cu_vien_cuoi_cung

# ==========================================================
# ===== VẼ KẾT QUẢ
# ==========================================================
def ve_bounding_boxes(image, contours, color=(0, 255, 0), thickness=2):
    image_result = image.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(image_result, (x, y), (x + w, y + h), color, thickness)
    return image_result

# ==========================================================
# ===== HÀM XỬ LÝ CHÍNH CHO MỖI ẢNH
# ==========================================================
def process_image(image_path: Path, output_dir: Path, config: dict):
    try:
        # --- BƯỚC 1: Đọc ảnh ---
        image_color = cv2.imread(str(image_path))
        if image_color is None:
            raise FileNotFoundError(f"Không thể tải ảnh: {image_path.name}")
        
        H, W = image_color.shape[:2]
        print(f"  Kích thước: {W}x{H}")

        # --- Lấy cấu hình ---
        BLUR_KERNEL_SIZE = config['BLUR_KERNEL_SIZE']
        CANNY_THRESHOLD_1 = config['CANNY_THRESHOLD_1']
        CANNY_THRESHOLD_2 = config['CANNY_THRESHOLD_2']
        CONNECT_KERNEL_SIZE = config['CONNECT_KERNEL_SIZE']
        CONNECT_DO_CLOSE = config['CONNECT_DO_CLOSE']
        CONNECT_DO_DILATE = config['CONNECT_DO_DILATE']
        MIN_AREA_RATIO = config['MIN_AREA_RATIO']
        MAX_AREA_RATIO = config['MAX_AREA_RATIO']
        MIN_AR_VUONG = config['MIN_AR_VUONG']
        MAX_AR_VUONG = config['MAX_AR_VUONG']
        MIN_AR_DAI = config['MIN_AR_DAI']
        MAX_AR_DAI = config['MAX_AR_DAI']

        # --- Tiền xử lý (Xám + Làm mờ) ---
        blur = tien_xu_ly(image_color) # Hàm này đã bao gồm cả cvtColor

        # --- Phát hiện cạnh Canny ---
        edges = cv2.Canny(blur, CANNY_THRESHOLD_1, CANNY_THRESHOLD_2, 
                         apertureSize=3, L2gradient=True)

        # --- Kết nối cạnh ---
        connected_image = ket_noi_canh(edges, kernel_size=CONNECT_KERNEL_SIZE, 
                                       shape='rect', 
                                       do_close=CONNECT_DO_CLOSE, 
                                       do_dilate=CONNECT_DO_DILATE)

        # --- Tìm tất cả contours ---
        contours_all, binary_used = tim_ung_cu_vien(connected_image)
        print(f"  Bước 5.1: Tìm thấy {len(contours_all)} contours ban đầu.")

        # --- Lọc theo diện tích ---
        min_area_px = int(MIN_AREA_RATIO * W * H)
        max_area_px = int(MAX_AREA_RATIO * W * H)
        
        contours_area_filtered = loc_theo_dien_tich(contours_all, min_area_px, max_area_px)
        print(f"  Bước 5.2: Lọc diện tích ({min_area_px}-{max_area_px} px) -> Còn lại {len(contours_area_filtered)}.")

        # --- Lọc theo tỷ lệ khung hình ---
        final_candidates = loc_theo_ty_le_khung_hinh(
            contours_area_filtered,
            min_aspect_ratio_vuong=MIN_AR_VUONG, max_aspect_ratio_vuong=MAX_AR_VUONG,
            min_aspect_ratio_dai=MIN_AR_DAI, max_aspect_ratio_dai=MAX_AR_DAI
        )
        print(f"  Bước 5.3: Lọc tỷ lệ -> Ứng cử viên cuối cùng: {len(final_candidates)}.")
        
        # --- Vẽ kết quả và Lưu ---
        image_final_result = ve_bounding_boxes(image_color, final_candidates, color=(0, 255, 0), thickness=2)
        
        # Tạo tên tệp đầu ra
        output_filename = f"{image_path.stem}_output.png"
        output_path = output_dir / output_filename
        
        # Lưu ảnh
        cv2.imwrite(str(output_path), image_final_result)
        print(f"  => Đã lưu kết quả tới: {output_path}")
        return True

    except FileNotFoundError as e:
        print(f"  LỖI: {e}")
        return False
    except ImportError:
        print("\n--- LỖI ---")
        print("Vui lòng cài đặt: pip install opencv-python numpy")
        # Dừng toàn bộ script nếu thiếu thư viện
        sys.exit(1)
    except Exception as e:
        print(f"  Đã xảy ra lỗi không xác định khi xử lý {image_path.name}: {e}")
        return False

# ==========================================================
# ===== HÀM CHÍNH: CHẠY TOÀN BỘ QUY TRÌNH HÀNG LOẠT
# ==========================================================

if __name__ == "__main__":
    
    # --- 0. Định nghĩa thư mục ---
    INPUT_DIR = Path("data_input")
    OUTPUT_DIR = Path("data_output")
    
    # --- 1. Cấu hình ---
    # Đưa tất cả cấu hình vào một dict để dễ dàng truyền đi
    config = {
        'BLUR_KERNEL_SIZE': (15, 15),
        'CANNY_THRESHOLD_1': 120,
        'CANNY_THRESHOLD_2': 240,
        'CONNECT_KERNEL_SIZE': (9, 5), # Tốt cho biển số (ngang)
        'CONNECT_DO_CLOSE': True,
        'CONNECT_DO_DILATE': True, # Dựa trên lệnh gọi hàm gốc
        'MIN_AREA_RATIO': 0.005, # 0.5% diện tích ảnh
        'MAX_AREA_RATIO': 0.08,  # 8% diện tích ảnh
        'MIN_AR_VUONG': 1.5, # Biển vuông (2 dòng)
        'MAX_AR_VUONG': 2.5,
        'MIN_AR_DAI': 3.5, # Biển dài (1 dòng)
        'MAX_AR_DAI': 5.0
    }
    
    # Tạo thư mục đầu ra nếu chưa tồn tại
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Đã tạo/xác nhận thư mục đầu ra: {OUTPUT_DIR.resolve()}")

    # Các đuôi tệp ảnh được phép
    allowed_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    
    print(f"--- BẮT ĐẦU XỬ LÝ HÀNG LOẠT TỪ THƯ MỤC: {INPUT_DIR.resolve()} ---")

    # Đảm bảo thư mục đầu vào tồn tại
    if not INPUT_DIR.exists():
        print(f"LỖI: Thư mục đầu vào không tồn tại: {INPUT_DIR.resolve()}")
        print("Vui lòng tạo thư mục 'data_input' và thêm ảnh vào đó.")
    else:
        # Lấy danh sách các tệp ảnh
        image_paths = [p for p in INPUT_DIR.iterdir() if p.suffix.lower() in allowed_extensions]
        total_files = len(image_paths)
        processed_count = 0
        
        if total_files == 0:
            print(f"Không tìm thấy ảnh nào (với đuôi {allowed_extensions}) trong {INPUT_DIR.resolve()}")
        else:
            print(f"Tìm thấy {total_files} tệp ảnh. Bắt đầu xử lý...")
            
            # Lặp qua từng ảnh và xử lý
            for image_path in image_paths:
                print(f"\n--- Xử lý tệp: {image_path.name} ---")
                success = process_image(image_path, OUTPUT_DIR, config)
                if success:
                    processed_count += 1
            
            print("\n--- XỬ LÝ HOÀN TẤT ---")
            print(f"Tổng số tệp đã xử lý thành công: {processed_count}/{total_files}")
    
    print("Chương trình kết thúc.")


Đã tạo/xác nhận thư mục đầu ra: C:\Users\nguye\handson-ml3\xu_ly_anh\D4-duong\data_output
--- BẮT ĐẦU XỬ LÝ HÀNG LOẠT TỪ THƯ MỤC: C:\Users\nguye\handson-ml3\xu_ly_anh\D4-duong\data_input ---
Tìm thấy 50 tệp ảnh. Bắt đầu xử lý...

--- Xử lý tệp: 1.jpg ---
  Kích thước: 475x192
  Bước 5.1: Tìm thấy 2 contours ban đầu.
  Bước 5.2: Lọc diện tích (456-7296 px) -> Còn lại 0.
  Bước 5.3: Lọc tỷ lệ -> Ứng cử viên cuối cùng: 0.
  => Đã lưu kết quả tới: data_output\1_output.png

--- Xử lý tệp: 10.jpg ---
  Kích thước: 435x280
  Bước 5.1: Tìm thấy 1 contours ban đầu.
  Bước 5.2: Lọc diện tích (609-9744 px) -> Còn lại 0.
  Bước 5.3: Lọc tỷ lệ -> Ứng cử viên cuối cùng: 0.
  => Đã lưu kết quả tới: data_output\10_output.png

--- Xử lý tệp: 11.jpg ---
  Kích thước: 400x566
  Bước 5.1: Tìm thấy 0 contours ban đầu.
  Bước 5.2: Lọc diện tích (1132-18112 px) -> Còn lại 0.
  Bước 5.3: Lọc tỷ lệ -> Ứng cử viên cuối cùng: 0.
  => Đã lưu kết quả tới: data_output\11_output.png

--- Xử lý tệp: 12.jpg ---
  Kích